# 🚀 Ready4Hire Fine-tuning en Google Colab

**IMPORTANTE**: Activar GPU en Colab
- Runtime → Change runtime type → Hardware accelerator → GPU (T4)

## Paso 1: Instalar dependencias

In [ ]:
# Instalar Unsloth
!pip install unsloth
!pip install --upgrade torch transformers datasets trl

## Paso 2: Subir archivos de dataset

Sube estos archivos desde tu computadora:
- `ready4hire_dataset_train.jsonl`
- `ready4hire_dataset_val.jsonl`

O conéctalos desde Google Drive

In [ ]:
# Montar Google Drive (opcional)
from google.colab import drive
drive.mount('/content/drive')

## Paso 3: Fine-tuning

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Configuración
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True

# Cargar modelo
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Aplicar LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("✓ Modelo cargado con LoRA")

In [ ]:
# Cargar dataset
dataset = load_dataset(
    "json",
    data_files={
        "train": "ready4hire_dataset_train.jsonl",
        "validation": "ready4hire_dataset_val.jsonl"
    }
)

print(f"Train: {len(dataset['train'])} ejemplos")
print(f"Val: {len(dataset['validation'])} ejemplos")

In [ ]:
# Formatear datos
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
# Entrenar
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
)

print("Iniciando entrenamiento...")
trainer_stats = trainer.train()
print("✓ Entrenamiento completado")

## Paso 4: Exportar modelo

In [ ]:
# Exportar a GGUF (formato Ollama)
model.save_pretrained_gguf(
    "ready4hire-finetuned",
    tokenizer,
    quantization_method="q4_k_m"
)

print("✓ Modelo exportado a GGUF")
print("Descarga el archivo .gguf y cópialo a: models/ready4hire-finetuned/")

## Paso 5: Descargar modelo

1. Descarga el archivo `.gguf` generado
2. Cópialo a tu computadora: `models/ready4hire-finetuned/`
3. Continúa con Fase 3: `python3 scripts/3_deployment/step1_import_to_ollama.py`